## TensorFlow CNN迁移学习

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 6.5 卷积神经网络迁移学习

●迁移学习：将一个问题上训练好的模型通过简单的调整使其适用于一个新的问题。  
●可以保留训练好的Inception-v3模型中所有卷积层的参数，只替换最后一层全连接层。最后这一层全连接层之前的网络层称之为**瓶颈层（bottleneck）**。有理由认为瓶颈层输出的节点向量可以被作为任何图像的一个更加精简且表达能力更强的特征向量。于是，在新数据集上，可以直接利用这个训练好的模型进行特征提取，再将这个特征向量作为输入来训练一个新的单层全连接神经网络处理新的分类问题。  
●一般来说，数据量足够的情况下，迁移学习的效果不如完全重新训练。但是迁移学习所需要的训练时间和训练样本都远远小于完整训练的模型。

1. 数据处理
下载将要用到的数据（218MB）：  
下载地址 http://download.tensorflow.org/example_images/flower_photos.tgz  
解压后含5个子文件夹，子文件夹名称为一种花的名称，代表不同类别。平均每种花734张RGB图片，大小不相同。

In [ ]:
# 下载flower_photos数据集
wget http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz

In [ ]:
import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

# 原始输入数据的目录，这个目录下有5个子目录，每个子目录下保存属于该类别的所有图片
INPUT_DATA = '/path/to/flower_photos'
# 输出文件地址。将整理后的图片数据通过numpy的格式保存
OUTPUT_FILE = '/path/to/flower_processed_data.npy'

# 测试数据和验证数据的比例
VALIDATION_PERCENTAGE = 10
TEST_PERCENTAGE = 10

# 测试数据并将数据分割成训练数据、验证数据和测试数据 
def create_image_lists(sess, testing_percentage, validation_percentage):
    sub_dirs = [x[0] for x in os.walk(INPUT_DATA)]
    is_root_dir = True
    
    # 初始化各个数据集
    training_image = []
    training_labels = []
    testing_image = []
    testing_labels = []
    validation_images = []
    validation_labels = []
    current_label = 0
    
    # 读取所有的子目录
    for sub_dir in sub_dirs:
        if is_root_dir:
            is_root_dir = False
            continue
            
    # 获取一个子目录中所有的图片文件
    extensions = ['jpg','jpeg','JPG','JPEG']
    file_list = []
    dir_name = os.path.basename(sub_dir)
    for extension in extensions:
        file_glob = os.path.join(INPUT_DATA, dir_name, '*.'+ extension)
        file_list.extend(glob.glob(file_glob))
        if not file_list: continue
        
        # 处理图片数据
        for file_name in file_list:
            # 读取并解析图片，将图片转化为299×299以便inception-v3模型来处理
            image_raw_data = gfile.FastGFile(file_name,'rb').read()
            image = tf.image.decode_jpeg(image_raw_data)
            if image.dtype != tf.float32:
                image = tf.image.convert_image_dtype(
                    image, dtype=tf.float32)
            # tf.image.resize_images可以重构像素数目
            image = tf.image.resize_images(image, [299, 299])
            image_value = sess.run(image)
            
            # 随机划分数据集
            chance = np.random.randint(100)
            if chance < validation_percentage:
                validation_images.append(image_value)
                validation_labels.append(current_label)
            elif chance < (testing_percentage + validation_percentage):
                testing_images.append(image_value)
                testing_labels.append(current_label)
            else:
                training_images.append(image_value)
                training_labels.append(current_label)
        current_label += 1
        
    # 将训练数据随机打乱以获得更好的训练效果
    state = np.random.get_state()
    np.random.shuffle(train_images)
    np.random.set_state(state)
    np.random.shuffle(training_labels)
    
    return np.asarray([training_images, training_labels,
                      validation_images, validation_labels,
                      testing_images, testing_labels])

# 数据整理主函数
def main():
    with tf.Session() as sess:
        processed_data = create_image_lists(
            sess, TEST_PERCENTAGE, VALIDATION_PERCENTAGE)
        # 通过numpy格式保存处理后的数据
        np.save(OUTPUT_FILE, processed_data)
        
if __name__ == '__main__':
    main()

上面代码将所有图片数据划分成了训练、验证、测试3个数据集，且将图片从原始jpg转化为299×299×3数字矩阵。  
下面下载谷歌提供的训练好的Inception-v3模型

In [ ]:
wget
http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz

#解压之后可以得到训练好的模型文件inception_v3.ckpt
tar xzf_inception_v3_2016_08_28.tar.gz

至此数据集和训练好的模型准备完毕。  
2. 迁移学习

In [ ]:
import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile
import tensorflow.contrib.slim as slim

# 加载通过TensorFlow-Slim定义好的inception_v3模型。
import tensorflow.contrib.slim.python.slim.nets.inception_v3 as inception_v3


# 1. 定义训练过程中将要使用到的常量。
# 处理好之后的数据文件。
INPUT_DATA = '../../datasets/flower_processed_data.npy'
# 保存训练好的模型的路径。这里可以将使用新数据训练得到的模型保存下来，如果计算充足，
# 还可以在训练完最后的全连接层之后再训练所有的网络层，使模型更贴近新数据。
TRAIN_FILE = 'train_dir/model'
# 谷歌提供的训练好的模型文件地址。
CKPT_FILE = '../../datasets/inception_v3.ckpt'

# 定义训练中使用的参数。
LEARNING_RATE = 0.0001
STEPS = 300
BATCH = 32
N_CLASSES = 5

# 不需要从谷歌训练好的模型中加载的参数。这里就是最后的全连接层，
# 因为在新的问题中要重新训练这一层中的参数。这里给出的是参数的前缀
CHECKPOINT_EXCLUDE_SCOPES = 'InceptionV3/Logits,InceptionV3/AuxLogits'
# 需要训练的网络层参数名称，在fine-tuning的过程中就是最后的全连接层。
# 这里给出的是参数的前缀
TRAINABLE_SCOPES = 'InceptionV3/Logits,InceptionV3/AuxLogit'


# 2. 获取所有需要从谷歌训练好的模型中加载的参数。
def get_tuned_variables():
    exclusions = [scope.strip() for scope in\
                  CHECKPOINT_EXCLUDE_SCOPES.split(',')]
    variables_to_restore = []
    # 枚举inception-v3模型中所有的参数，然后判断是否需要从加载列表中移除。
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)
    return variables_to_restore


# 3. 获取所有需要训练的变量列表。
def get_trainable_variables():    
    scopes = [scope.strip() for scope in TRAINABLE_SCOPES.split(',')]
    variables_to_train = []
    # 枚举所有需要训练的参数前缀，并通过这些前缀找到所有需要训练的参数。
    for scope in scopes:
        variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
        variables_to_train.extend(variables)
    return variables_to_train


# 4. 定义训练过程
def main():
    # 加载预处理好的数据。
    processed_data = np.load(INPUT_DATA)
    training_images = processed_data[0]
    n_training_example = len(training_images)
    training_labels = processed_data[1]
    
    validation_images = processed_data[2]
    validation_labels = processed_data[3]
    
    testing_images = processed_data[4]
    testing_labels = processed_data[5]
    print("%d training examples, %d validation examples and %d testing examples." % (
        n_training_example, len(validation_labels), len(testing_labels)))

    # 1. 定义inception-v3的输入输出
    images = tf.placeholder(tf.float32, [None, 299, 299, 3], name='input_images')
    labels = tf.placeholder(tf.int64, [None], name='labels')
    
    # 2. 定义前向传播、损失函数、反向传播
    # 定义inception-v3模型。因为谷歌给出的只有模型参数取值，所以这里
    # 需要在这个代码中定义inception-v3的模型结构。虽然理论上需要区分训练和
    # 测试中使用到的模型，也就是说在测试时应该使用is_training=False，但是
    # 因为预先训练好的inception-v3模型中使用的batch normalization参数与
    # 新的数据会有出入，所以这里直接使用同一个模型来做测试。
    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        logits, _ = inception_v3.inception_v3(
            images, num_classes=N_CLASSES, is_training=True)
    
    # 定义损失函数
    trainable_variables = get_trainable_variables()    # 自定义函数，获取需要训练的变量
    tf.losses.softmax_cross_entropy(
        tf.one_hot(labels, N_CLASSES), logits, weights=1.0)
    total_loss = tf.losses.get_total_loss()
    
    # 定义反向传播
    train_step = tf.train.RMSPropOptimizer(LEARNING_RATE).minimize(total_loss)
    
    # 计算正确率。
    with tf.name_scope('evaluation'):
        correct_prediction = tf.equal(tf.argmax(logits, 1), labels)
        evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                
    # 定义加载Google训练好的Inception-v3模型的Saver。
    load_fn = slim.assign_from_checkpoint_fn(
      CKPT_FILE,
      get_tuned_variables(),                          # 自定义函数，获取需要加载的参数
      ignore_missing_vars=True)
    
    # 3. 建立会话，训练模型
    saver = tf.train.Saver()      # 定义保存新模型的Saver。
    with tf.Session() as sess:
        # 初始化没有加载进来的变量。注意这个过程要在加载模型之前，
        # 否则初始化过程会将已经加载好的变量重新赋值。
        init = tf.global_variables_initializer()
        sess.run(init)
        
        # 加载谷歌已经训练好的模型。
        print('Loading tuned variables from %s' % CKPT_FILE)
        load_fn(sess)
            
        start = 0
        end = BATCH
        for i in range(STEPS):
            # 运行训练过程，这里不会更新全部参数，只会更新指定部分的参数
            _, loss = sess.run([train_step, total_loss], feed_dict={
                images: training_images[start:end], 
                labels: training_labels[start:end]})

            # 保存并输出日志
            if i % 30 == 0 or i + 1 == STEPS:
                saver.save(sess, TRAIN_FILE, global_step=i)
                validation_accuracy = sess.run(evaluation_step, feed_dict={
                    images: validation_images, labels: validation_labels})
                print('Step %d: Training loss is %.1f Validation accuracy = %.1f%%' % (
                    i, loss, validation_accuracy * 100.0))
                         
            # 获取下一个batch
            start = end
            if start == n_training_example:
                start = 0
            end = start + BATCH
            if end > n_training_example:
                end = n_training_example
            
        # 在最后的测试数据上测试正确率。
        test_accuracy = sess.run(evaluation_step, feed_dict={
            images: testing_images, labels: testing_labels})
        print('Final test accuracy = %.1f%%' % (test_accuracy * 100))
        
        
if __name__ == '__main__':
    main()

Final test accuracy 约为 91.9% ，可以看到模型在新的数据集上很快能够收敛，并达到不错的分类效果。  
通过这种特殊结构的神经网络，可以将图像识别问题的准确率提高到一个新的层次。